In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sharegpt-zh-1k/sharegpt_zh_1K_format.jsonl
/kaggle/input/test-txt/test0730.txt


In [3]:
!pip install peft==0.4.0
!pip install transformers==4.30.2
!pip install datasets==2.12.0
!pip install tqdm==4.65.0
!pip install loguru==0.7.0
!pip install fire==0.5.0
!pip install bitsandbytes==0.39.0
#wandb==0.15.3
!pip install cpm_kernels==1.0.11
!pip install accelerate==0.20.3
!pip install sentencepiece==0.1.99
!pip install deepspeed==0.9.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 7.9 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: datasets
    Found existing installation: datasets 2.1.0
    Uninstalling datasets-2.1.0:
      Successfully uninstalled datasets-2.1.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116949 sha256=c42a74e79eacedc3aa17b9e2d9d3a5b8964dc5ccc8cc8d124901b3c0a6ed4ac1
  Stored in directory: /root/.cache/pip/wheels/90/d4/f7/9404e5db0116bd4d43e5666eaa3e70ab53723e1e3ea40c9a95
Successfully built fire
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 MB 11.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.6/416.6 kB 7.3 MB/s eta 0:00:00a 0:00:01
     ━

In [4]:
import os
import argparse
from typing import List, Dict, Optional
from accelerate import init_empty_weights  # load an empty model,just structure , no real weight.
import bitsandbytes as bnb
import torch
from glob import glob
from loguru import logger
from datasets import load_dataset
from transformers import (
    AutoModel,
    AutoModelForCausalLM,
    AutoTokenizer,
    HfArgumentParser,
    set_seed,
    TrainingArguments,
    Trainer,
    BitsAndBytesConfig 
)
from peft import (
    TaskType,
    LoraConfig,
    #AdaLoraConfig ,  #  提出自2020年 感觉和lora区别不大 而且和qlora有冲突 这里代码没有用到 
                     #例子https://www.zhihu.com/question/596950521/answer/3109759716
    get_peft_model,
    set_peft_model_state_dict,
    prepare_model_for_kbit_training
)
from peft.utils import TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING
from transformers.deepspeed import HfDeepSpeedConfig
import deepspeed
import json
from itertools import chain

[2023-08-02 11:40:32,863] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so
/opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
CUDA SETUP: Loading binary /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so...


/opt/conda/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


In [5]:
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm2-6b", trust_remote_code=True)

A new version of the following files was downloaded from https://huggingface.co/THUDM/chatglm2-6b:
- tokenization_chatglm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


# 测试<font color = red>tokenizer.encode</font>方法中add_special_tokenizer=True/False的区别 
# 区别在于为True时会在tokenList首部添加2个特殊字符 id为64790和64792 二者对应的token是空白""

In [12]:
string_list ="我<unk></s>"
#string_list ="11+2+3+4"

print("add_special_tokens=True")
tokenized_res = tokenizer.encode(string_list,add_special_tokens=True,padding=True)
logger.info(f"add_special_tokens=True,tokenized_res={tokenized_res}")
print(tokenizer.batch_decode(tokenized_res,add_special_tokens=False))
print("add_special_tokens=False")
tokenized_res = tokenizer.encode(string_list,add_special_tokens=False,padding=True)
logger.info(f"add_special_tokens=False,tokenized_res={tokenized_res}")
print(tokenizer.batch_decode(tokenized_res,add_special_tokens=False))

2023-08-02 01:47:23.861 | INFO     | __main__:<module>:6 - add_special_tokens=True,tokenized_res=[64790, 64792, 34211, 31002, 3013, 6029, 30917, 30994]
2023-08-02 01:47:23.863 | INFO     | __main__:<module>:10 - add_special_tokens=False,tokenized_res=[34211, 31002, 3013, 6029, 30917, 30994]


add_special_tokens=True
['', '', '我', '<', 'unk', '></', 's', '>']
add_special_tokens=False
['我', '<', 'unk', '></', 's', '>']


# 那tokenizer.encode方法 和 <font color=green>tokenizer类方法是不是有什么区别呢？答案是肯定的
# 根据tokenization_chatglm.py 文件的https://huggingface.co/THUDM/chatglm2-6b/blob/main/tokenization_chatglm.py#L66 位置 model_input_names = ["input_ids", "attention_mask", "position_ids"]
# 所以最后会返回一个字典 里面是三个k-v， k分别是 ["input_ids", "attention_mask", "position_ids"]
# attention_mask 前面是 长度为padLen个0 后面全是1  所以其实代码中是left-pad
# posioton_ids 是从0开始的0，1，2，3，4，数值表示token位置
# input_ids 是token id 的list 
# 示例如下

In [13]:
string_list ="我<unk></s>"
#string_list ="11+2+3+4"


tokenized_res = tokenizer(string_list,add_special_tokens=True,padding=True)
logger.info(f"add_special_tokens=True,tokenized_res={tokenized_res}")
tokenized_res = tokenizer(string_list,add_special_tokens=False,padding=True)
logger.info(f"add_special_tokens=False,tokenized_res={tokenized_res}")

2023-08-02 01:58:22.260 | INFO     | __main__:<module>:6 - add_special_tokens=True,tokenized_res={'input_ids': [64790, 64792, 34211, 31002, 3013, 6029, 30917, 30994], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1], 'position_ids': [0, 1, 2, 3, 4, 5, 6, 7]}
2023-08-02 01:58:22.262 | INFO     | __main__:<module>:8 - add_special_tokens=False,tokenized_res={'input_ids': [34211, 31002, 3013, 6029, 30917, 30994], 'attention_mask': [1, 1, 1, 1, 1, 1], 'position_ids': [0, 1, 2, 3, 4, 5]}


# 根据上面的例子 我们发现 如果不需要attention 和 position_ids的话 其实使用tokenizer.encode方法就行了
# 处理起来更方便 我们来看一个对话的例子
# 注意chatglm的tokenizer.encode参数中默认bos和eos是没开启的 而且默认没有bos_token 但是有eos_token 
# 可能就根本没有用上这两个token

In [14]:
tokenizer.bos_token

Using bos_token, but it is not set yet.


In [15]:
tokenizer.eos_token

'</s>'

In [30]:


s1 = "1+2+3+4"
tk_s1 = tokenizer.encode(text=s1,add_special_tokens=False)
print(tk_s1,type(tk_s1),"len=",len(tk_s1))
s2 = "我也知道等于10"
tk_s2 = tokenizer.encode(text=s2,add_special_tokens=False)
print(tk_s2,type(tk_s2),"len=",len(tk_s2))
total = tk_s1 + tk_s2
print(total,"len=",len(total))
print(tokenizer.batch_decode(total,skip_special_tokens=True))

input_ids = total
labels = [-100] * len(tk_s1) +tk_s2

print(f"input_ids={input_ids}")
print(f"labels = {labels}")




[30910, 30939, 31011, 30943, 31011, 30966, 31011, 30972] <class 'list'> len= 8
[30910, 33876, 31717, 36738, 30939, 30940] <class 'list'> len= 6
[30910, 30939, 31011, 30943, 31011, 30966, 31011, 30972, 30910, 33876, 31717, 36738, 30939, 30940] len= 14
['', '1', '+', '2', '+', '3', '+', '4', '', '我也', '知道', '等于', '1', '0']
input_ids=[30910, 30939, 31011, 30943, 31011, 30966, 31011, 30972, 30910, 33876, 31717, 36738, 30939, 30940]
labels = [-100, -100, -100, -100, -100, -100, -100, -100, 30910, 33876, 31717, 36738, 30939, 30940]


In [38]:
# 试试多轮问答
Q_temp = "[Round {}]\n\n问：{}"
A_temp = "\n\n答：{}\n\n"
history=[("我今天想去玩术士。","你要打toc吗？"),("是的，我要买饰品。","老四的裁决？"),("怎么可能？我要买小强的亡者君临","哦哦对 裁决是物理毕业 不是SS的")]
input_ids =[]
labels =[]
for turn_number,conversation in enumerate(history):
    #print(turn_number + 1)
    q = conversation[0]
    a = conversation[1]
    #print(Q_temp.format(turn_number+1,q))
    #print(A_temp.format(a))
    Q = Q_temp.format(turn_number+1,q)
    A = A_temp.format(a)
    Q_token_list = tokenizer.encode(Q,add_special_tokens=False)
    A_token_list = tokenizer.encode(A,add_special_tokens=False)
    input_ids.extend(Q_token_list)
    input_ids.extend(A_token_list)
    labels.extend([-100]*len(Q_token_list))
    labels.extend(A_token_list)
print(f"input_ids={input_ids}")
print(f"labels={labels}")

input_ids_token_to_str = tokenizer.batch_decode(input_ids,skip_special_tokens=True)
labels_token_to_str = tokenizer.batch_decode(labels,skip_special_tokens=True)

print(input_ids_token_to_str)
print(labels_token_to_str)

input_ids=[790, 30951, 517, 30910, 30939, 30996, 13, 13, 54761, 31211, 54546, 31869, 37580, 55409, 54780, 54930, 31155, 30910, 13, 13, 55437, 31211, 34526, 54819, 30912, 420, 55398, 31514, 13, 13, 790, 30951, 517, 30910, 30943, 30996, 13, 13, 54761, 31211, 38586, 31123, 34034, 55304, 55739, 54667, 31155, 30910, 13, 13, 55437, 31211, 54729, 54795, 54530, 48568, 31514, 13, 13, 790, 30951, 517, 30910, 30966, 30996, 13, 13, 54761, 31211, 52687, 31514, 34034, 55304, 54603, 33026, 55737, 54631, 55835, 55290, 30910, 13, 13, 55437, 31211, 56645, 56645, 54570, 30910, 48568, 54532, 33176, 32121, 30910, 31663, 4800, 54530, 13, 13]
labels=[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 30910, 13, 13, 55437, 31211, 34526, 54819, 30912, 420, 55398, 31514, 13, 13, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 30910, 13, 13, 55437, 31211, 54729, 54795, 54530, 48568, 31514, 13, 13, -100, -100,

In [2]:
import json
with open("多轮对话数据.json","w",encoding="utf-8") as fw :
    dialogues =[
    {"history":[("我今天想去玩术士。","你要打toc吗？"),("是的，我要买饰品。","老四的裁决？"),("怎么可能？我要买小强的亡者君临","哦哦对 裁决是物理毕业 不是SS的")]},
    {"history":[("今天爆红","出裁决了？"),("是的，还出了浮冰长靴。","666啊老铁")]},
    {"history":[("今天icc爆红","出望远镜还是血毒了？")]},
    {"history":[("晚上25Hicc你要买啥？","我估计要存钱买望远镜，或者冰龙的饰品。")]},
    {"history":[("上周wcl我战士小萨打了1W4秒伤","那多少分了？"),("98.89","服务器第一就是你。")]},
    {"history":[("Htoc五十箱队伍来个橙分SM","电撒还是ZQS"),("我要奶萨","可以我奶元双休")]},
    {"history":[("你这个术士的装备真tm的壕，277毕业，连红玉的饰品都有。","这个号是我买的，淘宝上4000块。"),("我靠，有钱。","一般吧。")]}, 
    {"history":[("为啥我装等都264了 dps还这么低？","你是不是没卡药没开爆发？"),("我记得偷了药的，加速和烈焰菇都吃了","等会，我看你好像技能用的低等级的"),("啊？我去看看等级。。。。X 还真是","哈哈，亏你还打了这么久没发现，也是服了。")]}, 
    {"history":[("我看这个团长要黑G","才卖了2万就黑？"),("我看了下它装备，一身环保。感觉不靠谱啊。","我去问问它啥意思")]}, 
    {"history":[("啥时候开的新三本？","上周2"),("你们今天清了cd没？要不一起？","可以，等我换ms")]},     
    ]
    for item in dialogues:
        line = json.dumps(item,ensure_ascii=False)
        fw.write(line+"\n")
print(123)

123


In [3]:
import json
with open("sharegpt_format_conv.json",'w',encoding='utf-8') as fw :
    data = [
        {"conversations":[{"from":"human","value":"请为面向在线学习和个人辅导的新SaaS平台提供一些用于SEO的关键词。"},{"from":"gpt","value":"这里是一些关于新的SaaS平台的潜在SEO关键词，该平台专注于在线学习和私人辅导：\n* 在线学习平台\n* 私人辅导平台\n* E-learning解决方案\n* 辅导软件\n* 虚拟辅导\n* 在线辅导服务\n* 私人教育平台\n* 虚拟课堂软件\n* 远程学习平台\n* 在线教学工具"},{"from":"human","value":"如何利用这些关键词来扩大市场推广和认知？"},{"from":"gpt","value":"为了扩大市场覆盖率和提高认知度，您可以通过以下几种方式使用这些关键词：\n* 将它们融入到您的网站内容中，例如主页、产品页面和博客文章中。这将帮助搜索引擎了解您的网站内容，提高其在搜索结果中的排名。\n* 在您的网站元标题和元描述中使用它们，为潜在客户清晰地展示您的平台提供的服务。\n* 在社交媒体帖子、新闻通讯等营销材料中使用它们，以扩大受众范围。\n* 在付费广告活动中使用它们，例如 Google AdWords，以针对特定关键词并吸引正在寻找在线学习或私人辅导解决方案的潜在客户。\n* 优化您的网站以适应这些关键词，以提高它在搜索结果中的可见性，并使潜在客户更容易找到您。这可能包括优化您的网站结构、创建高质量内容以及从其他权威网站建立反向链接。"},{"from":"human","value":"给我一个使用这些关键词进行付费广告活动的策略。"},{"from":"gpt","value":"以下是在付费广告活动中使用这些关键词的潜在策略：\n1.从您提供的列表中识别最相关和高效的关键词。您可以使用关键词研究工具（如Google的Keyword Planner）来帮助您识别具有高搜索量和低竞争的关键词。\n2.将您的关键词组织成主题组或“广告组”。例如，您可以为与在线学习，私人教导和电子学习解决方案相关的关键词创建广告组。\n3.使用这些广告组创建针对性的广告，专门设计以吸引搜索这些关键词的潜在客户。例如，您可以为面向“虚拟课堂软件”和“远程学习平台”的关键词创建广告，以此来为您的在线学习平台打广告。\n4.设置您的广告以在搜索结果中出现，当潜在客户搜索这些关键词时能够看到您的广告。这将有助于确保您的广告在正确的时间被正确的人看到。\n5.监控您广告的表现，并根据需要调整您的关键词策略。这可能涉及添加新关键词，删除表现不佳的关键词或调整出价以提高广告的表现。\n通过遵循这一策略，您可以有效地利用关键词来吸引潜在客户，并在付费广告活动中提高您的SaaS平台的可见性。"}]},
        {"conversations":[{"from":"human","value":"谁是Soroush Dalili？请详细说明。"},{"from":"gpt","value":"Soroush Dalili是一名知名的网络安全研究员和顾问，拥有丰富的经验。他参与了许多与信息安全相关的高端项目和倡议，包括漏洞研究、渗透测试和Web应用安全。\nDalili是几本网络安全和Web应用安全的著作的作者，包括《保护ASP.NET Web应用程序》和《高级ASP.NET安全》。他还是备受欢迎的开源漏洞扫描工具“DotNetNuke Scanner”的创作者。\n除了作为作者和工具创作者的工作外，Dalili还是网络安全社区中备受追捧的演讲者和培训师。他在各种会议和活动中发表演讲和主持研讨会，包括黑帽大会、OWASP和Infosecurity Europe。\nDalili是负责任的披露和道德黑客实践的坚定倡导者。他的贡献得到了网络安全社区的认可，包括被授予微软MVP企业安全奖、ISC2信息安全领导奖和年度道德黑客奖。\n总的来说，Soroush Dalili是网络安全行业中备受尊敬和有影响力的人物，以他在Web应用安全领域的专业知识、对社区的贡献以及对该领域负责任和道德实践的承诺而闻名。"},{"from":"human","value":"你确定吗？其中许多事情是不准确或无效的。"},{"from":"gpt","value":"如果我提供的信息有任何错误或无效，我深表歉意。作为一个自然语言处理模型，我的回答基于训练数据集中可用的数据。如果有具体的错误点，请让我知道，我会尽力纠正。\n关于Soroush Dalili，我的先前回答是基于我知道的截至2021年9月的信息。如果他的职业或工作有任何最近的发展或更改，我可能不知道。"}]},
        {"conversations":[{"from":"human","value":"1800年代最显著的特征是什么？"},{"from":"gpt","value":"十九世纪又称为1800年代，是一个显著的文化、经济和政治变革的时期。1800年代的一些关键特点包括：\n1. 工业革命：1800年代是工业革命开始的时期，是一个工业化和技术进步迅速的时期。在此期间，新机器和工厂彻底改变了商品生产方式，促进了大规模生产和城市的增长。\n2. 政治和社会变革：1800年代有重大的政治和社会变革，包括许多国家废除奴隶制度、民主和选举权运动的扩张、民族主义和帝国主义的兴起等。\n3. 欧洲帝国的扩张：1800年代见证了欧洲帝国的不断扩张，欧洲列强试图扩大其在世界其他地区的影响力和控制力。这导致了许多非洲、亚洲和美洲地区被殖民，以及领土和资源的冲突。\n4. 科技进步：1800年代有重大的科技进步，包括蒸汽机、电报和铁路的发展。这些进步对交通和通讯产生了重大影响，并有助于塑造现代世界。\n5. 文化和艺术运动：1800年代也是文化和艺术创新的时期，浪漫主义、现实主义和印象主义在文学、艺术和音乐等领域中兴起。这些运动有助于塑造人们的思想和表达方式，并影响了20世纪艺术表现的发展。"},{"from":"human","value":"在19世纪，是蒸汽船还是火车对世界的影响更大？"},{"from":"gpt","value":"蒸汽船和火车在19世纪都具有重要影响力，因为它们都有助于在工业革命期间改革运输和通讯。\n\n蒸汽船由蒸汽引擎驱动，允许人和货物在河流和水道上进行运输。它们在贸易和商业的发展中发挥了关键作用，特别是在美国，并帮助开拓了新的地区以进行定居和扩张。\n\n火车也由蒸汽引擎驱动，可以快速运输人和货物。它们有助于连接城市和地区，在工业和商业的扩展中发挥了关键作用。火车还有助于促进城市的增长和城市中心的发展。\n\n总的来说，很难说蒸汽船和火车谁的影响更大，因为它们在19世纪的运输和通讯发展中都发挥了重要作用。但是，火车可能具有更广泛的影响，因为它们可以连接更多地区，便于更远距离货物的运输。"},{"from":"human","value":"火车是如何影响城市的？"},{"from":"gpt","value":"火车在19世纪及以后对城市产生了重大影响。火车对城市的影响包括：\n\n1. 促进城市的增长：火车使人们和货物快速运输成为可能，促进了城市的增长。随着城市通过铁路网与其它地区联系更加紧密，吸引了更多的居民和企业，导致城市中心扩张。\n\n2. 形塑城市发展：火车站和铁路线的建设对城市的布局和发展产生了重大影响。在许多情况下，城市围绕火车站发展起来，周边的商业和住宅也随之涌现。火车的存在还影响了工业的位置和货物在城市中的分配方式。\n\n3. 连接城市与更广泛的地区：火车帮助连接城市与更广泛的地区，使人们能够轻松地在城市之间和国家其他地方旅行。这促进了思想、货物和服务的交流，并有助于刺激经济增长。\n\n4. 改变了人们的生活和工作方式：火车对城市居民的生活和工作方式产生了重大影响。它们促进了货物和人员的快速运输，使人们更容易在城市的不同地区生活和工作。这导致了郊区的发展和通勤文化的增长。"},{"from":"human","value":"汽车在19世纪末和20世纪初对城市产生了什么影响？"},{"from":"gpt","value":"汽车在19世纪末20世纪初对城市产生了重大影响，随着汽车越来越普遍且价格逐渐变得更加实惠，人们的生活方式、工作方式等方面发生了重大变化。汽车对城市产生的影响包括：\n1. 改变人们的生活和工作方式：汽车让人们在城市内部和城市之间更加容易地进行出行，从而对人们的生活和工作产生了重大影响。人们更容易居住在郊区并通勤上下班，这引发了郊区化现象的发展和通勤文化的增长。\n2. 形塑城市的发展：汽车的日益普及导致城市规划和发展方式发生了变化。例如，城市开始优先建设道路和高速公路以容纳越来越多的汽车，停车成为城市规划中的重要考虑因素。\n3. 改变货物运输方式：汽车彻底改变了城市内外货物运输的方式。它们使得货物在短距离内能够快速运输，这导致一些货物的铁路运输逐渐减少。\n4. 影响环境：城市中汽车数量的增加也对环境产生了影响，因为燃烧化石燃料用于交通运输加剧了空气污染和气候变化。\n总的来说，在19世纪末20世纪初，汽车对城市的影响是非常显著的，它们改变了人们的生活和工作方式，塑造了城市的发展趋势，并对环境造成了影响。"},{"from":"human","value":"19世纪吊桥对城市的影响是什么？"},{"from":"gpt","value":"吊桥在19世纪对城市产生了显著的影响，因为它们允许建造长而坚固的桥梁，可以跨越宽阔的水体或山谷。吊桥对城市产生影响的方式包括：\n1. 连接城市：吊桥允许在河流和其他水体上建造桥梁，连接起原来被这些天然屏障隔绝的城市。这促进了物品，思想和人员之间的交流，并帮助刺激了经济增长。\n2. 塑造城市发展：吊桥的建造对城市的布局和发展产生了重大影响。在许多情况下，城市围绕桥梁建设的地点突显出，因为这些地区成为了贸易和商业的枢纽。\n3. 促进城市增长：吊桥允许城市跨越河流和其他水体的扩张，因为它们让人们和货物可以在两侧之间轻松地旅行。这促进了城市的增长以及新城区和街区的发展。\n4. 改善交通：吊桥也改善了城市内部和城市之间的交通，因为它们允许在河流和其他水体上建造道路和铁路。这使人和货物更容易移动，促进了贸易和商业的增长。\n总的来说，吊桥在19世纪对城市产生了重大影响，因为它们连接了城市，塑造了城市的发展，促进了城市的增长，并改善了交通。"},{"from":"human","value":"如果一个以19世纪为主题的城市建造游戏没有包含与火车或汽车有关的重要游戏元素，它会被认为是完整的吗？"},{"from":"gpt","value":"由城市建造游戏设计者决定他们希望在游戏中包括哪些游戏玩法元素。然而，在19世纪的城市建造游戏中，火车和汽车可能会被认为是重要的元素，因为它们都对城市的发展产生了重要影响。火车和汽车改变了人们的生活和工作方式，塑造了城市的布局和发展，并促进了商品和人员的交流。它们还在工业和商业的扩张以及城市中心的增长中起着关键性作用。也可以制作一个19世纪的城市建造游戏，其中不包括涉及火车或汽车的重要游戏玩法元素。在此类游戏中可以包括许多其他因素，如悬索桥的建设、工业和商业的发展以及人口和城市中心的增长。"},{"from":"human","value":"有意思。谢谢你的见解👍"},{"from":"gpt","value":"不客气，我很高兴能帮到你。如果你还有其他问题，请随时问我。"}]},

    ]
    for item in data :
        line = json.dumps(item,ensure_ascii=False)
        fw.write(line)
print(125)

125


In [7]:
cat 多轮对话数据.json

{"history": [["我今天想去玩术士。", "你要打toc吗？"], ["是的，我要买饰品。", "老四的裁决？"], ["怎么可能？我要买小强的亡者君临", "哦哦对 裁决是物理毕业 不是SS的"]]}
{"history": [["今天爆红", "出裁决了？"], ["是的，还出了浮冰长靴。", "666啊老铁"]]}
{"history": [["今天icc爆红", "出望远镜还是血毒了？"]]}
{"history": [["晚上25Hicc你要买啥？", "我估计要存钱买望远镜，或者冰龙的饰品。"]]}
{"history": [["上周wcl我战士小萨打了1W4秒伤", "那多少分了？"], ["98.89", "服务器第一就是你。"]]}
{"history": [["Htoc五十箱队伍来个橙分SM", "电撒还是ZQS"], ["我要奶萨", "可以我奶元双休"]]}
{"history": [["你这个术士的装备真tm的壕，277毕业，连红玉的饰品都有。", "这个号是我买的，淘宝上4000块。"], ["我靠，有钱。", "一般吧。"]]}
{"history": [["为啥我装等都264了 dps还这么低？", "你是不是没卡药没开爆发？"], ["我记得偷了药的，加速和烈焰菇都吃了", "等会，我看你好像技能用的低等级的"], ["啊？我去看看等级。。。。X 还真是", "哈哈，亏你还打了这么久没发现，也是服了。"]]}
{"history": [["我看这个团长要黑G", "才卖了2万就黑？"], ["我看了下它装备，一身环保。感觉不靠谱啊。", "我去问问它啥意思"]]}
{"history": [["啥时候开的新三本？", "上周2"], ["你们今天清了cd没？要不一起？", "可以，等我换ms"]]}


In [11]:
# 1000 sample too long don't print
#cat /kaggle/input/sharegpt-zh-1k/sharegpt_zh_1K_format.jsonl

In [5]:
3//2*2

2

In [15]:
my_dataset['train'].filter(lambda example : len(example["conversations"])>1)

Filter:   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset({
    features: ['conversations'],
    num_rows: 999
})

In [19]:
def tokenize_function_sharegpt(example,tokenizer):
    """
       多轮对话使用每个example（也就是一行json样本）中的example['history'] 拼接多轮对话 构建一个包含多轮对话的总的input_ids和总的labels
       这个Q_temp和A_temp 不同的model 都不一样 但是很重要 
       这里用chatglm2-6b官网的tokenization_chatglm.py中的
    """
    Q_temp = "[Round {}]\n\n问：{}"
    A_temp = "\n\n答：{}\n\n"
    input_ids =[]
    labels =[]
    must_be_even_len = len(example['conversations']) //2 *2
    if must_be_even_len == 0:
        print(f"must_be_even_len=0!")
        print(f"长度不足的example={example}")
        raise ValueError("example出现了长度不足2的异常。")
    #assert must_be_even_len > 0
    # must_be_even_len > 0 这个 条件由filter保证
    # my_dataset['train'].filter(lambda example : len(example["conversations"])>1)
    
    for idx in range(0,must_be_even_len,2):
        #print(turn_number + 1)
        q = example['conversations'][idx]['value']
        a = example['conversations'][idx+1]['value']
        #print(Q_temp.format(turn_number+1,q))
        #print(A_temp.format(a))
        Q = Q_temp.format(int(idx//2)+1,q)
        A = A_temp.format(a)
        Q_token_list = tokenizer.encode(Q,add_special_tokens=False)
        A_token_list = tokenizer.encode(A,add_special_tokens=False)
        input_ids.extend(Q_token_list)
        input_ids.extend(A_token_list)
        labels.extend([-100]*len(Q_token_list))
        labels.extend(A_token_list)
    #print(f"input_ids={input_ids}")
    #print(f"labels={labels}")
    assert len(input_ids) == len(labels) # 这里一定要保证长度相等 不然说明拼接出问题
    #input_ids_token_to_str = tokenizer.batch_decode(input_ids,skip_special_tokens=True)
    #labels_token_to_str = tokenizer.batch_decode(labels,skip_special_tokens=True)
    #print(input_ids_token_to_str)
    #print(labels_token_to_str)
    
    return({"input_ids":input_ids , "labels":labels})

In [13]:
from datasets import load_dataset
data_files_list = "/kaggle/input/sharegpt-zh-1k/sharegpt_zh_1K_format.jsonl"
my_dataset = load_dataset('json', data_files=data_files_list)
print(my_dataset)
#print(my_dataset['train']['conversations'])

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-11d1f3cfe85b31b5/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['conversations'],
        num_rows: 1000
    })
})


In [21]:
a="AlpA"
a.strip().lower()

'alpa'

In [18]:
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm2-6b", trust_remote_code=True)
tokenized_dataset = my_dataset['train'].filter(lambda example : len(example["conversations"])>1).map(
                                lambda example: tokenize_function_sharegpt(example, tokenizer=tokenizer),
                                batched=False, 
                                remove_columns=my_dataset['train'].column_names)
print(f"tokenized_dataset={tokenized_dataset}")
print(f"tokenizer.decode(tokenized_dataset[-1]['input_ids'],skip_special_tokens=False)=\n{tokenizer.decode(tokenized_dataset[-1]['input_ids'],skip_special_tokens=False)}")
print(f"tokenizer.decode(tokenized_dataset[-1]['labels'],skip_special_tokens=False)=\n{tokenizer.decode(tokenized_dataset[-1]['labels'],skip_special_tokens=False)}")
print(f"tokenized_dataset[-1]['input_ids']=\n{tokenized_dataset[-1]['input_ids']}")
print(f"tokenized_dataset[-1]['labels']=\n{tokenized_dataset[-1]['labels']}")
print(f"len(tokenized_dataset[-1]['input_ids']={len(tokenized_dataset[-1]['input_ids'])}")
print(f"len(tokenized_dataset[-1]['labels']={len(tokenized_dataset[-1]['labels'])}")

tokenized_dataset=Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 999
})
tokenizer.decode(tokenized_dataset[-1]['input_ids'],skip_special_tokens=False)=
[Round 1]

问：为一个3D引擎编写绑定C++到F#的测试案例。 

答：这是一个将C++ 3D引擎绑定到F#的样例测试用例：
测试用例标题：将C++ 3D引擎绑定到F#
测试目标：验证C++ 3D引擎成功地绑定到F#，并可以在F#环境中用于创建3D模型和场景。
测试步骤：
1. 创建一个具有创建3D模型和场景所需函数的C++ 3D引擎。
2. 使用F#Interop绑定将C++ 3D引擎绑定到F#环境。
3. 通过在F#环境中测试C++ 3D引擎函数来验证绑定过程是否成功。
期望结果：
* C++ 3D引擎函数在F#环境中应可用。
* 使用绑定的C++ 3D引擎，F#环境应能够创建3D模型和场景。
* 测试结果应与在C++环境中创建3D模型和场景的预期结果相匹配。


tokenizer.decode(tokenized_dataset[-1]['labels'],skip_special_tokens=False)=


答：这是一个将C++ 3D引擎绑定到F#的样例测试用例：
测试用例标题：将C++ 3D引擎绑定到F#
测试目标：验证C++ 3D引擎成功地绑定到F#，并可以在F#环境中用于创建3D模型和场景。
测试步骤：
1. 创建一个具有创建3D模型和场景所需函数的C++ 3D引擎。
2. 使用F#Interop绑定将C++ 3D引擎绑定到F#环境。
3. 通过在F#环境中测试C++ 3D引擎函数来验证绑定过程是否成功。
期望结果：
* C++ 3D引擎函数在F#环境中应可用。
* 使用绑定的C++ 3D引擎，F#环境应能够创建3D模型和场景。
* 测试结果应与在C++环境中创建3D模型和场景的预期结果相匹配。


tokenized_dataset[-1]['input_ids']=
[790, 30951, 517, 30910, 30939, 30996, 13, 13, 54761, 31211, 53713, 

In [14]:
# # load_data
# dialogue ={
#     "history":[("我今天想去玩术士。","你要打toc吗？"),("是的，我要买饰品。","老四的裁决？"),("怎么可能？我要买小强的亡者君临","哦哦对 裁决是物理毕业 不是SS的")],
# }
# # import json
# # j_dia=json.dumps(dialogue)
# # print(j_dia)
from datasets import load_dataset
data_files_list = "多轮对话数据.json"
my_dataset = load_dataset('json', data_files=data_files_list)
print(my_dataset)
print(my_dataset['train']['history'])

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-fbfec77ab37fdad4/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['history'],
        num_rows: 10
    })
})
[[['我今天想去玩术士。', '你要打toc吗？'], ['是的，我要买饰品。', '老四的裁决？'], ['怎么可能？我要买小强的亡者君临', '哦哦对 裁决是物理毕业 不是SS的']], [['今天爆红', '出裁决了？'], ['是的，还出了浮冰长靴。', '666啊老铁']], [['今天icc爆红', '出望远镜还是血毒了？']], [['晚上25Hicc你要买啥？', '我估计要存钱买望远镜，或者冰龙的饰品。']], [['上周wcl我战士小萨打了1W4秒伤', '那多少分了？'], ['98.89', '服务器第一就是你。']], [['Htoc五十箱队伍来个橙分SM', '电撒还是ZQS'], ['我要奶萨', '可以我奶元双休']], [['你这个术士的装备真tm的壕，277毕业，连红玉的饰品都有。', '这个号是我买的，淘宝上4000块。'], ['我靠，有钱。', '一般吧。']], [['为啥我装等都264了 dps还这么低？', '你是不是没卡药没开爆发？'], ['我记得偷了药的，加速和烈焰菇都吃了', '等会，我看你好像技能用的低等级的'], ['啊？我去看看等级。。。。X 还真是', '哈哈，亏你还打了这么久没发现，也是服了。']], [['我看这个团长要黑G', '才卖了2万就黑？'], ['我看了下它装备，一身环保。感觉不靠谱啊。', '我去问问它啥意思']], [['啥时候开的新三本？', '上周2'], ['你们今天清了cd没？要不一起？', '可以，等我换ms']]]


In [70]:
def tokenize_function(example,tokenizer):
    """
       多轮对话使用每个example（也就是一行json样本）中的example['history'] 拼接多轮对话 构建一个包含多轮对话的总的input_ids和总的labels
       这个Q_temp和A_temp 不同的model 都不一样 但是很重要 
       这里用chatglm2-6b官网的tokenization_chatglm.py中的
    """
    Q_temp = "[Round {}]\n\n问：{}"
    A_temp = "\n\n答：{}\n\n"
    input_ids =[]
    labels =[]
    for turn_number,conversation in enumerate(example['history']):
        #print(turn_number + 1)
        q = conversation[0]
        a = conversation[1]
        #print(Q_temp.format(turn_number+1,q))
        #print(A_temp.format(a))
        Q = Q_temp.format(turn_number+1,q)
        A = A_temp.format(a)
        Q_token_list = tokenizer.encode(Q,add_special_tokens=False)
        A_token_list = tokenizer.encode(A,add_special_tokens=False)
        input_ids.extend(Q_token_list)
        input_ids.extend(A_token_list)
        labels.extend([-100]*len(Q_token_list))
        labels.extend(A_token_list)
    #print(f"input_ids={input_ids}")
    #print(f"labels={labels}")
    assert len(input_ids) == len(labels) # 这里一定要保证长度相等 不然说明拼接出问题
    #input_ids_token_to_str = tokenizer.batch_decode(input_ids,skip_special_tokens=True)
    #labels_token_to_str = tokenizer.batch_decode(labels,skip_special_tokens=True)
    #print(input_ids_token_to_str)
    #print(labels_token_to_str)
    
    return({"input_ids":input_ids , "labels":labels})

In [71]:
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm2-6b", trust_remote_code=True)
tokenized_dataset = my_dataset['train'].map(
                                lambda example: tokenize_function(example, tokenizer=tokenizer),
                                batched=False, 
                                remove_columns=my_dataset['train'].column_names)
print(f"tokenized_dataset={tokenized_dataset}")
print(f"tokenizer.decode(tokenized_dataset[0]['input_ids'],skip_special_tokens=False)=\n{tokenizer.decode(tokenized_dataset[0]['input_ids'],skip_special_tokens=False)}")
print(f"tokenizer.decode(tokenized_dataset[0]['labels'],skip_special_tokens=False)=\n{tokenizer.decode(tokenized_dataset[0]['labels'],skip_special_tokens=False)}")
print(f"tokenized_dataset[0]['input_ids']=\n{tokenized_dataset[0]['input_ids']}")
print(f"tokenized_dataset[0]['labels']=\n{tokenized_dataset[0]['labels']}")
print(f"len(tokenized_dataset[0]['input_ids']={len(tokenized_dataset[0]['input_ids'])}")
print(f"len(tokenized_dataset[0]['labels']={len(tokenized_dataset[0]['labels'])}")

  0%|          | 0/10 [00:00<?, ?ex/s]

tokenized_dataset=Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 10
})
tokenizer.decode(tokenized_dataset[0]['input_ids'],skip_special_tokens=False)=
[Round 1]

问：我今天想去玩术士。 

答：你要打toc吗？

 [Round 2]

问：是的，我要买饰品。 

答：老四的裁决？

 [Round 3]

问：怎么可能？我要买小强的亡者君临 

答：哦哦对 裁决是物理毕业 不是SS的


tokenizer.decode(tokenized_dataset[0]['labels'],skip_special_tokens=False)=


答：你要打toc吗？

 

答：老四的裁决？

 

答：哦哦对 裁决是物理毕业 不是SS的


tokenized_dataset[0]['input_ids']=
[790, 30951, 517, 30910, 30939, 30996, 13, 13, 54761, 31211, 54546, 31869, 37580, 55409, 54780, 54930, 31155, 30910, 13, 13, 55437, 31211, 34526, 54819, 30912, 420, 55398, 31514, 13, 13, 790, 30951, 517, 30910, 30943, 30996, 13, 13, 54761, 31211, 38586, 31123, 34034, 55304, 55739, 54667, 31155, 30910, 13, 13, 55437, 31211, 54729, 54795, 54530, 48568, 31514, 13, 13, 790, 30951, 517, 30910, 30966, 30996, 13, 13, 54761, 31211, 52687, 31514, 34034, 55304, 54603, 33026, 55737, 54631, 55835, 55290, 30910, 13, 13, 55437, 31211, 56645, 56645, 545

In [72]:
def get_datset(data_path, tokenizer, max_samples=-1,global_args=None):
    """读取本地包含json/jsonl文件的目录，将目录中所有文件作为dataset，并tokenize，shuffle，返回datasets.dataset"""
    
    if not (data_path is not None and os.path.exists(data_path)):
        raise ValueError("data_path requires a directory pointing to   jsons/jsonls")
    """https://github.com/shibing624/MedicalGPT/blob/main/supervised_finetuning.py#L383"""
    data_files_list = glob(f'{data_path}/**/*.json', recursive=True) + glob(
                f'{data_path}/**/*.jsonl', recursive=True)
    logger.info(f"data files: {', '.join(data_files_list)}")
          
    data = load_dataset('json', data_files=data_files_list)
    ''' 只使用max_samples 个样本来参与train和eval  
        https://github.com/shibing624/MedicalGPT/blob/main/supervised_finetuning.py#L453
    '''
    logger.info(f"在取样之前 data len ={len(data['train'])}")
    if max_samples is not None and max_samples > 0:
            max_samples = min(len(data['train']), max_samples)  # 
            data['train'] =  data['train'].select(range(max_samples))
    logger.info(f"在取样之后 data len ={len(data['train'])}")
    
    column_names = data['train'].column_names  # remove_columns=column_names  ,remove all at once
    """tokenize_func 中是单样本处理的写法 所以这里的batched只能设置为False"""
    logger.info("preprocessing dataset...")
    
    tokenized_dataset = data['train'].map(
                                lambda example: tokenize_function(example, tokenizer=tokenizer),
                                batched=False, 
                                remove_columns=my_dataset['train'].column_names)
    print(f"tokenized_dataset={tokenized_dataset}")
    print(f"tokenizer.decode(tokenized_dataset[0]['input_ids'],skip_special_tokens=False)=\n{tokenizer.decode(tokenized_dataset[0]['input_ids'],skip_special_tokens=False)}")
    print(f"tokenizer.decode(tokenized_dataset[0]['labels'],skip_special_tokens=False)=\n{tokenizer.decode(tokenized_dataset[0]['labels'],skip_special_tokens=False)}")
    print(f"tokenized_dataset[0]['input_ids']=\n{tokenized_dataset[0]['input_ids']}")
    print(f"tokenized_dataset[0]['labels']=\n{tokenized_dataset[0]['labels']}")
    print(f"len(tokenized_dataset[0]['input_ids']={len(tokenized_dataset[0]['input_ids'])}")
    print(f"len(tokenized_dataset[0]['labels']={len(tokenized_dataset[0]['labels'])}")
    tokenized_dataset = tokenized_dataset.shuffle(seed = 42) # 在这里就打乱顺序了 后面就不用了
    tokenized_dataset = tokenized_dataset.flatten_indices()
    print("【乱序后】")
    print(f"tokenizer.decode(tokenized_dataset[0]['input_ids'],skip_special_tokens=False)=\n{tokenizer.decode(tokenized_dataset[0]['input_ids'],skip_special_tokens=False)}")
    print(f"tokenizer.decode(tokenized_dataset[0]['labels'],skip_special_tokens=False)=\n{tokenizer.decode(tokenized_dataset[0]['labels'],skip_special_tokens=False)}")
    return tokenized_dataset

In [73]:
class DataCollatorForChatGLM:
    def __init__(self,
                 pad_token_id: int,
                 max_length: int = 2048,
                 ignore_label_id: int = -100):
        self.pad_token_id = pad_token_id
        self.ignore_label_id = ignore_label_id
        self.max_length = max_length

    def __call__(self, batch_data: List[Dict[str, List]]) -> Dict[str, torch.Tensor]:
        """根据batch最大长度做padding
           注意 input_ids 已经是所有轮次对话Q+A+Q+A的拼接了 而且长度等于labels
           将list_of_{dict(input_ids,labels)} -> dict_of(input_ids:list  ,labels:list)
           所以叫做collator 收集 聚集
        """
        len_list = [len(d['input_ids']) for d in batch_data]
        batch_max_len = max(len_list)
        input_ids, labels = [], []
        for len_of_d, d in sorted(zip(len_list, batch_data), key=lambda x: -x[0]):
            pad_len = batch_max_len - len_of_d
            ids = d['input_ids'] + [self.pad_token_id] * pad_len
            label = d['labels'] + [self.ignore_label_id] * pad_len  # 注意这里是在右边padding 而且labels使用的是ignore_token_id
            if batch_max_len > self.max_length:
                ids = ids[: self.max_length]
                label = label[: self.max_length]
            input_ids.append(torch.LongTensor(ids))
            labels.append(torch.LongTensor(label))
        input_ids = torch.stack(input_ids)
        labels = torch.stack(labels)
        return {'input_ids': input_ids, 'labels': labels}

In [77]:
ds = get_datset(data_path="/kaggle/working/multi_turn", tokenizer=tokenizer, max_samples=-1,global_args=None)

2023-08-02 07:14:39.316 | INFO     | __main__:get_datset:9 - data files: /kaggle/working/multi_turn/多轮对话数据.json


  0%|          | 0/1 [00:00<?, ?it/s]

2023-08-02 07:14:40.161 | INFO     | __main__:get_datset:15 - 在取样之前 data len =10
2023-08-02 07:14:40.163 | INFO     | __main__:get_datset:19 - 在取样之后 data len =10
2023-08-02 07:14:40.164 | INFO     | __main__:get_datset:23 - preprocessing dataset...


  0%|          | 0/10 [00:00<?, ?ex/s]

tokenized_dataset=Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 10
})
tokenizer.decode(tokenized_dataset[0]['input_ids'],skip_special_tokens=False)=
[Round 1]

问：我今天想去玩术士。 

答：你要打toc吗？

 [Round 2]

问：是的，我要买饰品。 

答：老四的裁决？

 [Round 3]

问：怎么可能？我要买小强的亡者君临 

答：哦哦对 裁决是物理毕业 不是SS的


tokenizer.decode(tokenized_dataset[0]['labels'],skip_special_tokens=False)=


答：你要打toc吗？

 

答：老四的裁决？

 

答：哦哦对 裁决是物理毕业 不是SS的


tokenized_dataset[0]['input_ids']=
[790, 30951, 517, 30910, 30939, 30996, 13, 13, 54761, 31211, 54546, 31869, 37580, 55409, 54780, 54930, 31155, 30910, 13, 13, 55437, 31211, 34526, 54819, 30912, 420, 55398, 31514, 13, 13, 790, 30951, 517, 30910, 30943, 30996, 13, 13, 54761, 31211, 38586, 31123, 34034, 55304, 55739, 54667, 31155, 30910, 13, 13, 55437, 31211, 54729, 54795, 54530, 48568, 31514, 13, 13, 790, 30951, 517, 30910, 30966, 30996, 13, 13, 54761, 31211, 52687, 31514, 34034, 55304, 54603, 33026, 55737, 54631, 55835, 55290, 30910, 13, 13, 55437, 31211, 56645, 56645, 545

Flattening the indices:   0%|          | 0/1 [00:00<?, ?ba/s]

【乱序后】
tokenizer.decode(tokenized_dataset[0]['input_ids'],skip_special_tokens=False)=
[Round 1]

问：Htoc五十箱队伍来个橙分SM 

答：电撒还是ZQS

 [Round 2]

问：我要奶萨 

答：可以我奶元双休


tokenizer.decode(tokenized_dataset[0]['labels'],skip_special_tokens=False)=


答：电撒还是ZQS

 

答：可以我奶元双休




tokenizer.pad_token_id =0  pad_token=<unk>
    eos_token_id=2 eos_token=</s>

In [75]:
for item in ds:
    #print(item)
    print(tokenizer.decode(item['input_ids'],skip_special_tokens=False))
    print("======================")

[Round 1]

问：晚上25Hicc你要买啥？ 

答：我估计要存钱买望远镜，或者冰龙的饰品。


[Round 1]

问：上周wcl我战士小萨打了1W4秒伤 

答：那多少分了？

 [Round 2]

问：98.89 

答：服务器第一就是你。


[Round 1]

问：今天icc爆红 

答：出望远镜还是血毒了？


[Round 1]

问：为啥我装等都264了 dps还这么低？ 

答：你是不是没卡药没开爆发？

 [Round 2]

问：我记得偷了药的，加速和烈焰菇都吃了 

答：等会，我看你好像技能用的低等级的

 [Round 3]

问：啊？我去看看等级。。。。X 还真是 

答：哈哈，亏你还打了这么久没发现，也是服了。


[Round 1]

问：你这个术士的装备真tm的壕，277毕业，连红玉的饰品都有。 

答：这个号是我买的，淘宝上4000块。

 [Round 2]

问：我靠，有钱。 

答：一般吧。


[Round 1]

问：今天爆红 

答：出裁决了？

 [Round 2]

问：是的，还出了浮冰长靴。 

答：666啊老铁


[Round 1]

问：Htoc五十箱队伍来个橙分SM 

答：电撒还是ZQS

 [Round 2]

问：我要奶萨 

答：可以我奶元双休


[Round 1]

问：我今天想去玩术士。 

答：你要打toc吗？

 [Round 2]

问：是的，我要买饰品。 

答：老四的裁决？

 [Round 3]

问：怎么可能？我要买小强的亡者君临 

答：哦哦对 裁决是物理毕业 不是SS的




In [78]:
# 验证get_dataset
ds = get_datset(data_path="/kaggle/working/multi_turn", tokenizer=tokenizer, max_samples=-1,global_args=None)
for item in ds:
    #print(item)
    print(tokenizer.decode(item['input_ids'],skip_special_tokens=False))
    print("======================")
# 验证 DataCollatorForChatGLM
very_clear_data_collator = DataCollatorForChatGLM(pad_token_id=tokenizer.pad_token_id,max_length=900)
very_clear_data_collator(ds)


{'input_ids': tensor([[  790, 30951,   517,  ..., 31155,    13,    13],
         [  790, 30951,   517,  ...,     0,     0,     0],
         [  790, 30951,   517,  ...,     0,     0,     0],
         ...,
         [  790, 30951,   517,  ...,     0,     0,     0],
         [  790, 30951,   517,  ...,     0,     0,     0],
         [  790, 30951,   517,  ...,     0,     0,     0]]),
 'labels': tensor([[ -100,  -100,  -100,  ..., 31155,    13,    13],
         [ -100,  -100,  -100,  ...,  -100,  -100,  -100],
         [ -100,  -100,  -100,  ...,  -100,  -100,  -100],
         ...,
         [ -100,  -100,  -100,  ...,  -100,  -100,  -100],
         [ -100,  -100,  -100,  ...,  -100,  -100,  -100],
         [ -100,  -100,  -100,  ...,  -100,  -100,  -100]])}

In [9]:
logger.info(f"tokenizer.pad_token_id={tokenizer.pad_token_id},tokenizer.pad_token={tokenizer.pad_token}")

2023-07-31 11:01:16.541 | INFO     | __main__:<module>:1 - tokenizer.pad_token_id=0,tokenizer.pad_token=<unk>


In [10]:
logger.info(f"tokenizer.pad_token_id={tokenizer.eos_token_id},tokenizer.pad_token={tokenizer.eos_token}")

2023-07-31 11:01:19.595 | INFO     | __main__:<module>:1 - tokenizer.pad_token_id=2,tokenizer.pad_token=</s>


In [11]:
logger.info(f"tokenizer.pad_token_id={tokenizer.bos_token_id},tokenizer.pad_token={tokenizer.bos_token}")

Using bos_token, but it is not set yet.
2023-07-31 11:01:22.910 | INFO     | __main__:<module>:1 - tokenizer.pad_token_id=None,tokenizer.pad_token=None


In [114]:
def get_dataset(data_path, tokenizer, global_args=None,max_samples=None):
    """读取本地包含json/jsonl文件的目录，将目录中所有文件作为dataset，并tokenize，shuffle，返回datasets.dataset"""
    
    if not (data_path is not None and os.path.exists(data_path)):
        raise ValueError("data_path requires a directory pointing to  .txt files")
    """https://github.com/shibing624/MedicalGPT/blob/main/supervised_finetuning.py#L383"""
    data_files_list = glob(f'{data_path}/**/*.txt', recursive=True) 
    logger.info(f"data files: {', '.join(data_files_list)}")
          
    extension = "text"
    # sasmple_by="document" 就是整个txt全部读取了
    raw_datasets = load_dataset(
        extension,
        data_files=data_files_list,
        sample_by="document"
    )
#     logger.info(f"在取样之前 data len ={len(data['train'])}")
#     if max_samples is not None and max_samples > 0:
#             max_samples = min(len(data['train']), max_samples)  # 
#             data['train'] =  data['train'].select(range(max_samples))
#     logger.info(f"在取样之后 data len ={len(data['train'])}")
    
#     column_names = data['train'].column_names  # remove_columns=column_names  ,remove all at once
#     """tokenize_func 中是单样本处理的写法 所以这里的batched只能设置为False"""
#     logger.info("preprocessing dataset...")
#     dataset = data['train'].map(lambda example: tokenize_func(example, tokenizer, global_args),
#                                 batched=False, 
#                                 remove_columns=column_names)
#     dataset = dataset.shuffle(seed=global_args.seed)
#     dataset = dataset.flatten_indices()
    return raw_datasets

In [115]:
def tokenize_function(examples,tokenizer):
    # 这个函数必须返回dict 
    ## 不然raw_datasets.map(tokenize_function, ...)调用会提示报错
    ## 要么就直接这么写  return tokenizer(examples["text"])
    return {"input_ids" :tokenizer(examples["text"]).input_ids}

In [116]:
block_size = 10

In [117]:
def group_texts(examples,block_size):
        # Concatenate all texts.
        #concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
        total_length = len(examples[list(examples.keys())[0]])
        # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
        print(f"total_length={total_length}")
        
        #ADD 20230731 原先shiming是去掉了最后的尾巴 但是我觉得可以保留 毕竟语料不多 每一部分都要用上
        #if total_length >= block_size:
        #    total_length = (total_length // block_size) * block_size
        
        # Split by chunks of max_len.
        result = {
            k: [t[i: i + block_size] for i in range(0, total_length, block_size)]
            for k, t in examples.items()
        }
        #result["labels"] = result["input_ids"].copy()
        return result

In [118]:
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm2-6b", trust_remote_code=True)

In [119]:
pad_token_id = tokenizer.pad_token_id
print(f"pad_token_id = {pad_token_id}")

pad_token_id = 0


In [121]:
import random
def get_chained_lm_datasets(lm_datasets,
                            pad_token_id: int ,
                            global_args_max_length: int = 2048,
                            num_samples=-1,
                            tokenizer=None):
    "做padding和根据global.max_length截取"
    input_ids_list = list(chain(*lm_datasets['input_ids']))
    # padding  with max_len_of_samples
    max_len_ids = max([len(input_ids) for input_ids in input_ids_list] ) 
    print(f"样本中ids 最大长度 max_len_ids={max_len_ids}")
    print(f"全局训练参数的global_args_max_length={global_args_max_length}")
    print(f"cutoff to len = {min(global_args_max_length,max_len_ids)}")
    
    #截断
    new_input_ids=[]
    for ids in input_ids_list :
        print(f"len_sample_ids={len(ids)}")
        pad_len = max_len_ids - len(ids)
        ids += [pad_token_id]* pad_len
        # 截取 根据命令行输入的max len和 ids_list中最大长度 两值做比较取较小的来截取前面N个
        ids =ids[: min(global_args_max_length,max_len_ids)]
        new_input_ids.append(ids)
    # 抽样
    random_chosen_samples=new_input_ids
    print(f"验证查看 shuffle之前 第一个样本\n{tokenizer.decode(random_chosen_samples[0])}")
    print(f"验证查看 shuffle之前 最后一个样本\n{tokenizer.decode(random_chosen_samples[-1])}")
    random.shuffle(new_input_ids)
    if num_samples > -1: # num_samples  一般只是debug的时候取少量样本测试 ；
        #random.sample 不重复抽样
        random_chosen_samples = random.sample(new_input_ids,k=num_samples)
    print(f"样本数量={len(random_chosen_samples)}")
    #return {"input_ids":random_chosen_samples  ,"labels":random_chosen_samples.copy()}
    return [{"input_ids": item ,"labels":item.copy() } for item in random_chosen_samples]

In [92]:
def fault_tolerance_data_collator(features: List) -> Dict[str, Any]:
    if not isinstance(features[0], Mapping):
        features = [vars(f) for f in features]
    first = features[0]
    batch = {}

    # Special handling for labels.
    # Ensure that tensor is created with the correct type
    if "label" in first and first["label"] is not None:
        label = first["label"].item() if isinstance(first["label"], torch.Tensor) else first["label"]
        dtype = torch.long if isinstance(label, int) else torch.float
        batch["labels"] = torch.tensor([f["label"] for f in features], dtype=dtype)
    elif "label_ids" in first and first["label_ids"] is not None:
        if isinstance(first["label_ids"], torch.Tensor):
            batch["labels"] = torch.stack([f["label_ids"] for f in features])
        else:
            dtype = torch.long if type(first["label_ids"][0]) is int else torch.float
            batch["labels"] = torch.tensor([f["label_ids"] for f in features], dtype=dtype)

    # Handling of all other possible keys.
    # Again, we will use the first element to figure out which key/values are not None for this model.
    try:
        for k, v in first.items():
            if k not in ("label", "label_ids") and v is not None and not isinstance(v, str):
                if isinstance(v, torch.Tensor):
                    batch[k] = torch.stack([f[k] for f in features])
                elif isinstance(v, np.ndarray):
                    batch[k] = torch.tensor(np.stack([f[k] for f in features]))
                else:
                    batch[k] = torch.tensor([f[k] for f in features])
    except ValueError:  # quick fix by simply take the first example
        for k, v in first.items():
            if k not in ("label", "label_ids") and v is not None and not isinstance(v, str):
                if isinstance(v, torch.Tensor):
                    batch[k] = torch.stack([features[0][k]] * len(features))
                elif isinstance(v, np.ndarray):
                    batch[k] = torch.tensor(np.stack([features[0][k]] * len(features)))
                else:
                    batch[k] = torch.tensor([features[0][k]] * len(features))
    #print("打印fault_tolerance_data_collator处理后的信息")
    #print(f"batch.keys()={batch.keys()},len_batch_key_labels={len(batch['labels'])}    ,len_batch_key_k={len(batch[k])}")
    return batch

In [93]:
a=[1,2,3]
random.sample(a,k=2)

[3, 2]

In [122]:
# 是两段文本
raw_datasets = get_dataset(data_path="/kaggle/working/test",tokenizer=tokenizer,max_samples=10000)
print(raw_datasets)
print(raw_datasets['train']['text'])

2023-07-31 14:05:20.255 | INFO     | __main__:get_dataset:8 - data files: /kaggle/working/test/test/test0730.txt, /kaggle/working/test/test/test0730_2.txt


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 2
    })
})
["1+2+3+466666 66666 hgsg sgs hshshs hsfh shsh sh dh fhh\n4+3+2=1                     afghu wfsvjoer n gw jgohuth4odjf ajaddpjfwon sdjnfslf  \n我很优秀  你呢？我不知道啊，哈哈。。。 vsoro nvlknlbh h rwlj flvnskd gb gd \n吃放睡觉打豆豆哦，【】[];'lvsvd a  gg  \\n brh5p bph owgwk rwf s59jbg\nhd g  ghr5 eg r hsghrwgrwhhwh hg  wfsvjoer\ng shth h j efwe gb\n\nef g h hsr \n\n\nge whreh \n hgs ghhth", "1+2+3+466666 66666 hgsg sgs hshshs hsfh shsh sh dh fhh\n4+3+2=1                     afghu wfsvjoer n gw jgohuth4odjf ajaddpjfwon sdjnfslf  \n我很优秀  你呢？我不知道啊，哈哈。。。 vsoro nvlknlbh h rwlj flvnskd gb gd \n吃放睡觉打豆豆哦，【】[];'lvsvd a  gg  \\n brh5p bph owgwk rwf s59jbg\nhd g  ghr5 eg r hsghrwgrwhhwh hg  wfsvjoer\ng shth h j efwe gb\n\nef g h hsr \n\n\nge whreh \n hgs ghhth"]


In [124]:
tokenized_datasets = raw_datasets.map(
                lambda examples:tokenize_function(examples,tokenizer),
                batched=True,
                num_proc=2,#data_args.preprocessing_num_workers,
                remove_columns=['text'],#column_names,
                #load_from_cache_file=not data_args.overwrite_cache,
                desc="Running tokenizer on dataset",
            )
print(tokenized_datasets)
print(tokenized_datasets['train']['input_ids'])

DatasetDict({
    train: Dataset({
        features: ['input_ids'],
        num_rows: 2
    })
})
[[64790, 64792, 30910, 30939, 31011, 30943, 31011, 30966, 31011, 30972, 30978, 30978, 30978, 30978, 30978, 30910, 30978, 30978, 30978, 30978, 30978, 299, 6196, 30927, 268, 6196, 299, 1480, 1480, 30917, 299, 10240, 30919, 438, 1480, 438, 292, 30919, 279, 12578, 13, 30972, 31011, 30966, 31011, 30943, 30980, 30939, 647, 296, 10789, 30927, 11408, 271, 9563, 30933, 5282, 266, 310, 317, 30929, 467, 30927, 1063, 1056, 30972, 374, 30965, 30926, 29953, 2065, 30925, 30965, 30926, 23347, 268, 30921, 30965, 30916, 30926, 5239, 30926, 265, 13, 36545, 31994, 265, 54622, 55282, 31514, 42660, 55674, 31123, 34380, 39507, 5088, 14724, 310, 30933, 30920, 16462, 26604, 30919, 299, 416, 30929, 30920, 30965, 795, 30933, 30916, 2823, 30921, 317, 30931, 317, 30921, 30910, 13, 55058, 54804, 35228, 54819, 55720, 55720, 56645, 31123, 31538, 31529, 30995, 4571, 30953, 30920, 30933, 13532, 30921, 260, 265, 1298, 265, 

In [243]:
#raw_data['train']['text']

In [128]:
block_size=50

In [129]:
lm_datasets = tokenized_datasets['train'].map(
                lambda examples :group_texts(examples ,block_size) ,
                batched=False,
                num_proc=2,#data_args.preprocessing_num_workers,
                #load_from_cache_file=not data_args.overwrite_cache,
                desc=f"Grouping texts in chunks of {block_size}",
                #remove_columns =['attention_mask', 'position_ids',]
            )
print(lm_datasets)

Grouping texts in chunks of 50 (num_proc=2):   0%|          | 0/2 [00:00<?, ? examples/s]

total_length=219total_length=219

Dataset({
    features: ['input_ids'],
    num_rows: 2
})


In [130]:
dataset_train = get_chained_lm_datasets(lm_datasets,
                        pad_token_id=tokenizer.pad_token_id,
                        global_args_max_length=50000,
                         num_samples=-1,
                        tokenizer = tokenizer)
print(dataset_train)

样本中ids 最大长度 max_len_ids=50
全局训练参数的global_args_max_length=50000
cutoff to len = 50
len_sample_ids=50
len_sample_ids=50
len_sample_ids=50
len_sample_ids=50
len_sample_ids=19
len_sample_ids=50
len_sample_ids=50
len_sample_ids=50
len_sample_ids=50
len_sample_ids=19
shuffle之前 第一个样本 =1+2+3+466666 66666 hgsg sgs hshshs hsfh shsh sh dh fhh
4+3+2=1                    
shuffle之前 最后一个样本 =h hsr 


ge whreh 
 hgs ghhth
样本数量=10
[{'input_ids': [299, 299, 30917, 30918, 30910, 13, 13, 13, 465, 366, 263, 30919, 30910, 13, 299, 6196, 317, 12578, 398, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': [299, 299, 30917, 30918, 30910, 13, 13, 13, 465, 366, 263, 30919, 30910, 13, 299, 6196, 317, 12578, 398, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}, {'input_ids': [10789, 30927, 11408, 271, 9563, 30933, 5282, 266, 310, 317, 30929, 467, 30927, 1063, 1056, 30972, 374, 30965, 30926, 29953, 2065, 30925, 30965,

In [68]:
dataset_eval = get_chained_lm_datasets(lm_datasets,
                        pad_token_id=tokenizer.pad_token_id,
                        global_args_max_length=5,
                                 num_samples=2)
print(dataset_eval)

样本中ids 最大长度 max_len_ids=219
全局训练参数的global_args_max_length=5
cutoff to len = 5
样本数量=2
[{'input_ids': [64790, 64792, 30910, 30939, 31011], 'labels': [64790, 64792, 30910, 30939, 31011]}, {'input_ids': [64790, 64792, 30910, 30939, 31011], 'labels': [64790, 64792, 30910, 30939, 31011]}]


In [33]:
def get_datset_for_pretrain(data_path, tokenizer, block_size=10,global_args_max_length=0,max_samples=0):
    """读取本地包含json/jsonl文件的目录，将目录中所有文件作为dataset，并tokenize，shuffle，返回datasets.dataset"""
    print(123)
    if not (data_path is not None and os.path.exists(data_path)):
        raise ValueError("data_path requires a directory pointing to  .txt files")
    """https://github.com/shibing624/MedicalGPT/blob/main/supervised_finetuning.py#L383"""
    data_files_list = glob(f'{data_path}/**/*.txt', recursive=True) 
    logger.info(f"data files: {', '.join(data_files_list)}")
          
    extension = "text"
    # 读取全部txt文档 有N篇文档 那raw_datasets这个list就包含N个篇章级别的文本结果
    #sasmple_by="document" 就是将一个txt当做一个sample全部读取
    raw_datasets = load_dataset(
        extension,
        data_files=data_files_list,
        sample_by="document"
    )
    # 对全部N个文档str 做分词 分词结果也是N个长list 每个list都含有input_ids这个
    tokenized_datasets = raw_datasets.map(
                lambda examples : tokenize_function(examples ,tokenizer),
                batched=True,
                num_proc=2,#data_args.preprocessing_num_workers,
                remove_columns=['text'],#column_names,
                #load_from_cache_file=not data_args.overwrite_cache,
                desc="Running tokenizer on dataset",
            )
    # N个长list在这一步被按照block_size切成小段 原先是[长1],[长2] 现在仍然为2元素 但是每个里面都是小段[[短1-1],[短1-2]] ,[[短2-1],[短2-2]]
    lm_datasets = tokenized_datasets['train'].map(
                lambda examples :group_texts(examples , block_size),
                batched=False,
                num_proc=2,#data_args.preprocessing_num_workers,
                #load_from_cache_file=not data_args.overwrite_cache,
                #desc=f"Grouping texts in chunks of {block_size}",
                #remove_columns =['attention_mask', 'position_ids',]
            )
    dataset_final = get_chained_lm_datasets(lm_datasets,
                        pad_token_id=tokenizer.pad_token_id,
                        global_args_max_length=global_args_max_length,
                        num_samples=max_samples,
                        tokenizer = tokenizer)
#     logger.info(f"在取样之前 data len ={len(data['train'])}")
#     if max_samples is not None and max_samples > 0:
#             max_samples = min(len(data['train']), max_samples)  # 
#             data['train'] =  data['train'].select(range(max_samples))
#     logger.info(f"在取样之后 data len ={len(data['train'])}")
    
#     column_names = data['train'].column_names  # remove_columns=column_names  ,remove all at once
#     """tokenize_func 中是单样本处理的写法 所以这里的batched只能设置为False"""
#     logger.info("preprocessing dataset...")
#     dataset = data['train'].map(lambda example: tokenize_func(example, tokenizer, global_args),
#                                 batched=False, 
#                                 remove_columns=column_names)
#     dataset = dataset.shuffle(seed=global_args.seed)
#     dataset = dataset.flatten_indices()
    return dataset_final

In [34]:
train_ds = get_datset_for_pretrain(data_path="/kaggle/working/test",
                                  tokenizer =  AutoTokenizer.from_pretrained("THUDM/chatglm2-6b", trust_remote_code=True),
                                  block_size = 10,
                                  global_args_max_length=6,
                                  max_samples=-1)
print(train_ds)

2023-07-31 11:08:57.939 | INFO     | __main__:get_datset_for_pretrain:8 - data files: /kaggle/working/test/test/test0730.txt, /kaggle/working/test/test/test0730_2.txt


123


  0%|          | 0/1 [00:00<?, ?it/s]

Running tokenizer on dataset (num_proc=2):   0%|          | 0/2 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/2 [00:00<?, ? examples/s]

total_length=219total_length=219

样本中ids 最大长度 max_len_ids=10
全局训练参数的global_args_max_length=6
cutoff to len = 6
样本数量=42
[{'input_ids': [13, 15939, 317, 265, 30927, 5323], 'labels': [13, 15939, 317, 265, 30927, 5323]}, {'input_ids': [30929, 30934, 416, 30929, 30926, 268], 'labels': [30929, 30934, 416, 30929, 30926, 268]}, {'input_ids': [30974, 30916, 822, 30919, 30970, 30925], 'labels': [30974, 30916, 822, 30919, 30970, 30925]}, {'input_ids': [2065, 30925, 30965, 30926, 23347, 268], 'labels': [2065, 30925, 30965, 30926, 23347, 268]}, {'input_ids': [10789, 30927, 11408, 271, 9563, 30933], 'labels': [10789, 30927, 11408, 271, 9563, 30933]}, {'input_ids': [30929, 9563, 30933, 5282, 266, 13], 'labels': [30929, 9563, 30933, 5282, 266, 13]}, {'input_ids': [10038, 5323, 30929, 689, 2277, 30919], 'labels': [10038, 5323, 30929, 689, 2277, 30919]}, {'input_ids': [54804, 35228, 54819, 55720, 55720, 56645], 'labels': [54804, 35228, 54819, 55720, 55720, 56645]}, {'input_ids': [2065, 30925, 30965, 309

In [35]:
eval_ds = get_datset_for_pretrain(data_path="/kaggle/working/test",
                                  tokenizer =  AutoTokenizer.from_pretrained("THUDM/chatglm2-6b", trust_remote_code=True),
                                  block_size = 10,
                                  global_args_max_length=6,
                                  max_samples=5)
print(eval_ds)

2023-07-31 11:09:06.787 | INFO     | __main__:get_datset_for_pretrain:8 - data files: /kaggle/working/test/test/test0730.txt, /kaggle/working/test/test/test0730_2.txt


123


  0%|          | 0/1 [00:00<?, ?it/s]

样本中ids 最大长度 max_len_ids=10
全局训练参数的global_args_max_length=6
cutoff to len = 6
样本数量=5
[{'input_ids': [42660, 55674, 31123, 34380, 39507, 5088], 'labels': [42660, 55674, 31123, 34380, 39507, 5088]}, {'input_ids': [299, 10240, 30919, 438, 1480, 438], 'labels': [299, 10240, 30919, 438, 1480, 438]}, {'input_ids': [10789, 30927, 11408, 271, 9563, 30933], 'labels': [10789, 30927, 11408, 271, 9563, 30933]}, {'input_ids': [30929, 467, 30927, 1063, 1056, 30972], 'labels': [30929, 467, 30927, 1063, 1056, 30972]}, {'input_ids': [13, 30972, 31011, 30966, 31011, 30943], 'labels': [13, 30972, 31011, 30966, 31011, 30943]}]


In [38]:
import math
import os
from dataclasses import dataclass, field
from glob import glob
from itertools import chain
from typing import Optional, List, Dict, Any, Mapping
def fault_tolerance_data_collator(features: List) -> Dict[str, Any]:
    if not isinstance(features[0], Mapping):
        features = [vars(f) for f in features]
    first = features[0]
    batch = {}

    # Special handling for labels.
    # Ensure that tensor is created with the correct type
    if "label" in first and first["label"] is not None:
        label = first["label"].item() if isinstance(first["label"], torch.Tensor) else first["label"]
        dtype = torch.long if isinstance(label, int) else torch.float
        batch["labels"] = torch.tensor([f["label"] for f in features], dtype=dtype)
    elif "label_ids" in first and first["label_ids"] is not None:
        if isinstance(first["label_ids"], torch.Tensor):
            batch["labels"] = torch.stack([f["label_ids"] for f in features])
        else:
            dtype = torch.long if type(first["label_ids"][0]) is int else torch.float
            batch["labels"] = torch.tensor([f["label_ids"] for f in features], dtype=dtype)

    # Handling of all other possible keys.
    # Again, we will use the first element to figure out which key/values are not None for this model.
    try:
        for k, v in first.items():
            if k not in ("label", "label_ids") and v is not None and not isinstance(v, str):
                if isinstance(v, torch.Tensor):
                    batch[k] = torch.stack([f[k] for f in features])
                elif isinstance(v, np.ndarray):
                    batch[k] = torch.tensor(np.stack([f[k] for f in features]))
                else:
                    batch[k] = torch.tensor([f[k] for f in features])
    except ValueError:  # quick fix by simply take the first example
        for k, v in first.items():
            if k not in ("label", "label_ids") and v is not None and not isinstance(v, str):
                if isinstance(v, torch.Tensor):
                    batch[k] = torch.stack([features[0][k]] * len(features))
                elif isinstance(v, np.ndarray):
                    batch[k] = torch.tensor(np.stack([features[0][k]] * len(features)))
                else:
                    batch[k] = torch.tensor([features[0][k]] * len(features))
    #print("打印fault_tolerance_data_collator处理后的信息")
    #print(f"batch.keys()={batch.keys()},len_batch_key_labels={len(batch['labels'])}    ,len_batch_key_k={len(batch[k])}")
    return batch

In [39]:
fault_tolerance_data_collator(eval_ds)

{'input_ids': tensor([[42660, 55674, 31123, 34380, 39507,  5088],
         [  299, 10240, 30919,   438,  1480,   438],
         [10789, 30927, 11408,   271,  9563, 30933],
         [30929,   467, 30927,  1063,  1056, 30972],
         [   13, 30972, 31011, 30966, 31011, 30943]]),
 'labels': tensor([[42660, 55674, 31123, 34380, 39507,  5088],
         [  299, 10240, 30919,   438,  1480,   438],
         [10789, 30927, 11408,   271,  9563, 30933],
         [30929,   467, 30927,  1063,  1056, 30972],
         [   13, 30972, 31011, 30966, 31011, 30943]])}

In [40]:
class DataCollatorForChatGLM:
    
    # 虽然这个函数中又做了pad和截断 但是实际在pt阶段没有效果 因为之前已经是统一长度而且把尾部短文本用
    # pad token 补齐过了 这里代码留着是为了后面参考改多轮对话用
    def __init__(self,
                 pad_token_id: int,
                 max_length: int = 2048,
                 ignore_label_id: int = -100):
        self.pad_token_id = pad_token_id
        self.ignore_label_id = ignore_label_id
        self.max_length = max_length

    def __call__(self, batch_data: List[Dict[str, List]]) -> Dict[str, torch.Tensor]:
        """根据batch最大长度做padding"""
        len_list = [len(d['input_ids']) for d in batch_data]
        batch_max_len = max(len_list)
        input_ids, labels = [], []
        for len_of_d, d in sorted(zip(len_list, batch_data), key=lambda x: -x[0]):
            #padding :虽然预训练之前的步骤做了 长度已经是统一的 但是这个可以继续pad
            #注意 ids和label使用的pad token不同 
            # 在预训练阶段 下面的6行实际没有任何作用 因为pad_len 此时为0 可以打印看看
            pad_len = batch_max_len - len_of_d
            print(f"pad_len in data collator = {pad_len}")
            ids = d['input_ids'] + [self.pad_token_id] * pad_len
            label = d['labels'] + [self.ignore_label_id] * pad_len
            if batch_max_len > self.max_length:
                ids = ids[: self.max_length]
                label = label[: self.max_length]
            input_ids.append(torch.LongTensor(ids))
            labels.append(torch.LongTensor(label))
        input_ids = torch.stack(input_ids)
        labels = torch.stack(labels)
        return {'input_ids': input_ids, 'labels': labels}

data_collator = DataCollatorForChatGLM(pad_token_id=tokenizer.pad_token_id,
                                           max_length=7)

In [41]:
data_collator(train_ds)

{'input_ids': tensor([[   13, 15939,   317,   265, 30927,  5323],
         [30929, 30934,   416, 30929, 30926,   268],
         [30974, 30916,   822, 30919, 30970, 30925],
         [ 2065, 30925, 30965, 30926, 23347,   268],
         [10789, 30927, 11408,   271,  9563, 30933],
         [30929,  9563, 30933,  5282,   266,    13],
         [10038,  5323, 30929,   689,  2277, 30919],
         [54804, 35228, 54819, 55720, 55720, 56645],
         [ 2065, 30925, 30965, 30926, 23347,   268],
         [30978,   299,  6196, 30927,   268,  6196],
         [30929, 30934,   416, 30929, 30926,   268],
         [  299,   299, 30917, 30918, 30910,    13],
         [30916,  2823, 30921,   317, 30931,   317],
         [16462, 26604, 30919,   299,   416, 30929],
         [30978,   299,  6196, 30927,   268,  6196],
         [30978, 30978, 30978, 30978, 30978, 30910],
         [64790, 64792, 30910, 30939, 31011, 30943],
         [   13, 30972, 31011, 30966, 31011, 30943],
         [  467,   315, 30926,   

In [250]:
# 注意加上*
a=lm_datasets['input_ids']
list(chain(*a))


[[64790, 64792, 30910, 30939, 31011, 30943, 31011, 30966, 31011, 30972],
 [30978, 30978, 30978, 30978, 30978, 30910, 30978, 30978, 30978, 30978],
 [30978, 299, 6196, 30927, 268, 6196, 299, 1480, 1480, 30917],
 [299, 10240, 30919, 438, 1480, 438, 292, 30919, 279, 12578],
 [13, 30972, 31011, 30966, 31011, 30943, 30980, 30939, 647, 296],
 [10789, 30927, 11408, 271, 9563, 30933, 5282, 266, 310, 317],
 [30929, 467, 30927, 1063, 1056, 30972, 374, 30965, 30926, 29953],
 [2065, 30925, 30965, 30926, 23347, 268, 30921, 30965, 30916, 30926],
 [5239, 30926, 265, 13, 36545, 31994, 265, 54622, 55282, 31514],
 [42660, 55674, 31123, 34380, 39507, 5088, 14724, 310, 30933, 30920],
 [16462, 26604, 30919, 299, 416, 30929, 30920, 30965, 795, 30933],
 [30916, 2823, 30921, 317, 30931, 317, 30921, 30910, 13, 55058],
 [54804, 35228, 54819, 55720, 55720, 56645, 31123, 31538, 31529, 30995],
 [4571, 30953, 30920, 30933, 13532, 30921, 260, 265, 1298, 265],
 [30974, 30916, 822, 30919, 30970, 30925, 280, 670, 20092,

In [138]:
lm_datasets['train']['input_ids']

[[[64790, 64792, 30910, 30939, 31011, 30943, 31011, 30966, 31011, 30972],
  [30978, 30978, 30978, 30978, 30978, 30910, 30978, 30978, 30978, 30978],
  [30978, 299, 6196, 30927, 268, 6196, 299, 1480, 1480, 30917],
  [299, 10240, 30919, 438, 1480, 438, 292, 30919, 279, 12578],
  [13, 30972, 31011, 30966, 31011, 30943, 30980, 30939, 647, 296],
  [10789, 30927, 11408, 271, 9563, 30933, 5282, 266, 310, 317],
  [30929, 467, 30927, 1063, 1056, 30972, 374, 30965, 30926, 29953],
  [2065, 30925, 30965, 30926, 23347, 268, 30921, 30965, 30916, 30926],
  [5239, 30926, 265, 13, 36545, 31994, 265, 54622, 55282, 31514],
  [42660, 55674, 31123, 34380, 39507, 5088, 14724, 310, 30933, 30920],
  [16462, 26604, 30919, 299, 416, 30929, 30920, 30965, 795, 30933],
  [30916, 2823, 30921, 317, 30931, 317, 30921, 30910, 13, 55058],
  [54804, 35228, 54819, 55720, 55720, 56645, 31123, 31538, 31529, 30995],
  [4571, 30953, 30920, 30933, 13532, 30921, 260, 265, 1298, 265],
  [30974, 30916, 822, 30919, 30970, 30925, 2

In [173]:
a=[[64790, 64792, 30910, 30939, 31011, 30943, 31011, 30966, 31011, 30972],
  [30978, 30978, 30978, 30978, 30978, 30910, 30978, 30978, 30978, 30978]]
b=[[30978, 299, 6196, 30927, 268, 6196, 299, 1480, 1480, 30917],
  [299, 10240, 30919, 438, 1480, 438, 292, 30919, 279, 12578],
  [13, 30972, 31011, 30966, 31011, 30943, 30980, 30939, 647, 296],]
from itertools import chain
list(chain(a,b))

[[64790, 64792, 30910, 30939, 31011, 30943, 31011, 30966, 31011, 30972],
 [30978, 30978, 30978, 30978, 30978, 30910, 30978, 30978, 30978, 30978],
 [30978, 299, 6196, 30927, 268, 6196, 299, 1480, 1480, 30917],
 [299, 10240, 30919, 438, 1480, 438, 292, 30919, 279, 12578],
 [13, 30972, 31011, 30966, 31011, 30943, 30980, 30939, 647, 296]]

In [133]:
# 可以验证
#list(chain(lm_datasets['train']['input_ids'][0] ,lm_datasets['train']['input_ids'][1]))

In [134]:
#可以验证
#tokenized_dataset['train']['input_ids']

In [399]:
a=[1,2]
a[:5]

[1, 2]

In [1]:
!git clone https://github.com/valkryhx/chatGLM-6B-QLoRA

Cloning into 'chatGLM-6B-QLoRA'...
remote: Enumerating objects: 941, done.
remote: Counting objects: 100% (161/161), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 941 (delta 123), reused 73 (delta 73), pack-reused 780
Receiving objects: 100% (941/941), 17.26 MiB | 15.56 MiB/s, done.
Resolving deltas: 100% (592/592), done.


In [100]:
cd /kaggle/working/chatGLM-6B-QLoRA

/kaggle/working/chatGLM-6B-QLoRA


In [ ]:
!git pull --all --force
#!pip install peft==0.4.0
#!pip install  -U git+https://github.com/huggingface/peft.git
%cd /kaggle/working/chatGLM-6B-QLoRA 
!ls
!pip install -r requirements.txt
#!pip install deepspeed==0.9.5  这个也是需要的 但是目前kaggle 的runtime自带了

In [110]:
ls data/pretrain/xuanyan

2  xy.txt


In [149]:
!git pull --all --force 
#!pip install  -U git+https://github.com/huggingface/peft.git   # 20230717 peft==0.4.0正式发布了 不用调版本了推理完后再训练需要重新升级到0.4.0dev 所以有这句
!deepspeed --include localhost:0,1  pretrain_qlora_chatglm2.py \
  --train_args_json luzi.json \
  --model_name_or_path THUDM/chatglm2-6b \
  --output_dir output-pt \
  --num_train_samples -1 \
  --num_eval_samples 20 \
  --block_size 1024 \
  --train_data_path ./data/pretrain/xuanyan/ \
  --eval_data_path  ./data/pretrain/xuanyan/ \
  --max_length 1024 \
  --lora_rank 64 \
  --lora_dropout 0.05 \
  --compute_dtype fp16 \
  --per_device_train_batch_size 1 \
  --per_device_eval_batch_size 1  \
  --gradient_accumulation_steps 1 \
  --learning_rate  5e-5 \
  --num_train_epochs  40  \
  --save_total_limit 2 \
  --load_in_4bit True \
--deepspeed ds_zero2_config.json

Fetching origin
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), 682 bytes | 227.00 KiB/s, done.
From https://github.com/valkryhx/chatGLM-6B-QLoRA
   5d7feba..78dd514  main       -> origin/main
Updating 5d7feba..78dd514
Fast-forward
 pretrain_qlora_chatglm2.py | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)
[2023-07-31 15:42:59,920] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unabl

# 多轮对话sft 

In [1]:
!git clone https://github.com/valkryhx/chatGLM-6B-QLoRA

fatal: destination path 'chatGLM-6B-QLoRA' already exists and is not an empty directory.


In [2]:
!git pull --all --force
#!pip install peft==0.4.0
#!pip install  -U git+https://github.com/huggingface/peft.git
%cd /kaggle/working/chatGLM-6B-QLoRA 
!ls
!pip install -r requirements.txt
#!pip install deepspeed==0.9.5  这个也是需要的 但是目前kaggle 的runtime自带了

fatal: not a git repository (or any parent up to mount point /kaggle)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
/kaggle/working/chatGLM-6B-QLoRA
README.md		    pretrain_qlora_chatglm2.py
chatGLM_6B_QLoRA.json	    qa_aug.json
data			    qlora_param.json
ds_zero2_config.json	    remote_scripts
ds_zero3_config.json	    requirements.txt
inference_qlora.py	    state.db
inference_test.py	    train_ds_zero2_test.py——错误的
luzi.json		    train_normal_lora.py
merge_lora_and_quantize.py  train_qlora.py
output-pt		    train_qlora_deepspeed_zero.py
pics			    参考与经验
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9

In [5]:
!git pull --all --force 
#!pip install  -U git+https://github.com/huggingface/peft.git   # 20230717 peft==0.4.0正式发布了 不用调版本了推理完后再训练需要重新升级到0.4.0dev 所以有这句
!deepspeed --include localhost:0,1  sft_qlora_chatglm2.py  \
  --train_args_json luzi.json \
  --model_name_or_path THUDM/chatglm2-6b \
  --output_dir output-multi-turn-sft-0802-v1 \
  --num_train_samples -1 \
  --num_eval_samples 5 \
  --train_data_path ./data/multi_turn_conversations   \
  --eval_data_path  ./data/multi_turn_conversations     \
  --max_length 1024 \
  --lora_rank 64 \
  --lora_dropout 0.05 \
  --compute_dtype fp16 \
  --per_device_train_batch_size 1 \
  --per_device_eval_batch_size 1  \
  --gradient_accumulation_steps 1 \
  --learning_rate  5e-5 \
  --num_train_epochs  40  \
  --save_total_limit 2 \
  --load_in_4bit True \
--deepspeed ds_zero2_config.json

Fetching origin
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), 675 bytes | 337.00 KiB/s, done.
From https://github.com/valkryhx/chatGLM-6B-QLoRA
   3d29fa8..290f0bd  main       -> origin/main
Updating 3d29fa8..290f0bd
Fast-forward
 sft_qlora_chatglm2.py | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)
[2023-08-02 08:13:42,194] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to 